In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Reading in and preprocessing data

In [ ]:
# Reading in data from the 50-50 readmit vs. nonreadmit dataset

df = pd.read_csv('dfd.csv')
df.head()

In [ ]:
# Dropping certain columns

# Dropping ID numbers and dates
df = df.drop(columns=['subject_id', 'hadm_id', 'admittime', 'dischtime'])

# Dropping labevents and chartevents values of less importance based on feature selection (2/3 values for each measurement)
df = df.drop(columns=['rdw_min', 'rdw_max', 'hemoglobin_min', 'hemoglobin_max', 'creatinine_median', 'creatinine_min', 
                      'hematocrit_median', 'hematocrit_min', 'tempc_median', 'tempc_max', 'resprate_median', 
                      'resprate_min', 'wbc_median', 'wbc_max', 'inr_min', 'inr_median', 'ptt_median', 'ptt_max', 
                      'lactate_median', 'lactate_max', 'sysbp_median', 'sysbp_min', 'spo2_median', 'spo2_max', 
                      'bilirubin_median', 'bilirubin_max', 'platelet_median', 'platelet_max', 'heartrate_min',
                      'heartrate_median'])
df.head()

In [ ]:
# Converting categorical features into dummy variables

df_converted = pd.get_dummies(df)
df_converted.head()

In [ ]:
# Splitting dataframe into data (predictors) vs. label (attributed to be predicted)

label_df = df_converted.pop('followed_by_readmit')
data_df = df_converted
print('label_df:\n', label_df.head(), 2*'\n', 'data_df:\n', data_df.head())

In [6]:
# Converting dataframes to NumPy arrays

label = label_df.values
data = data_df.values

In [7]:
label

array([ True,  True,  True, ..., False, False, False])

In [8]:
data

array([[70, 8, 0, ..., 0, 0, 0],
       [42, 19, 0, ..., 0, 0, 0],
       [60, 8, 0, ..., 0, 1, 0],
       ...,
       [74, 19, 0, ..., 0, 0, 0],
       [67, 7, 0, ..., 0, 0, 0],
       [59, 9, 0, ..., 0, 0, 0]], dtype=object)

## Train/test split

In [9]:
# 80/20 train-test split

from sklearn.model_selection import train_test_split

train_data, test_data, train_label, test_label = train_test_split(data, label, train_size=0.8, test_size=0.2, random_state=10)

print('Training data:', train_data.shape, '\tTest data:', test_data.shape)
print('Training labels:', train_label.shape, '\tTest labels:', test_label.shape)

Training data: (4873, 110) 	Test data: (1219, 110)
Training labels: (4873,) 	Test labels: (1219,)


In [10]:
# Cross-validation on the training set (no need to do explicitly for Logistic Reg. since has a cv parameter, but just in case...)

#from sklearn.model_selection import KFold
#kf = KFold(n_splits=10, random_state=10)
#for train, test in kf.split(train_data, train_label):
    #print(test)

Note that cross-validation will be performed with the training data for each machine learning algorithm tested below.

## (Penalized) Logistic regression

From http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression:
"LogisticRegressionCV implements Logistic Regression with builtin cross-validation to find out the optimal C parameter (similar to what GridSearchCV might do)."

In [11]:
from sklearn.linear_model import LogisticRegressionCV

First, let's try using the `liblinear` solver (recommended for small datasets) with 'l1' penalty:

In [12]:
# Instantiating logistic regression estimator object; cv=10 indicates 10-fold cross-validation

lrcv_l1 = LogisticRegressionCV(cv=10, penalty='l1', solver='liblinear', random_state=10)

In [13]:
%%time

lrcv_l1.fit(train_data, train_label)

Wall time: 5min 40s


LogisticRegressionCV(Cs=10, class_weight=None, cv=10, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', random_state=10,
           refit=True, scoring=None, solver='liblinear', tol=0.0001,
           verbose=0)

In [14]:
lrcv_l1_pred = lrcv_l1.predict(test_data)
lrcv_l1_pp = lrcv_l1.predict_proba(test_data)

Running self.classes_ on the result from fitting the model to the data (see commented code below) tells us that the probability estimates for positive class ("True") are in column 1 rather than column 0 of the result from `predict_proba()` - this is what we'll need to use when calculating the ROC-AUC.

In [15]:
#lrcv_l1.fit(train_data_sc, train_label).classes_

In [16]:
from sklearn.metrics import roc_auc_score

lrcv_l1_probs = lrcv_l1.predict_proba(test_data)[:,1]
print('ROC-AUC score with l1 penalty:', roc_auc_score(test_label, lrcv_l1_probs))

ROC-AUC score with l1 penalty: 0.7012404754976451


What if we used 'l2' penalty instead?

In [17]:
%%time
# Repeating the code above for logistic regression but with l2 penalty

lrcv_l2 = LogisticRegressionCV(cv=10, penalty='l2', solver='liblinear', random_state=10)
lrcv_l2.fit(train_data, train_label)
lrcv_l2_pred = lrcv_l2.predict(test_data)
lrcv_l2_pp = lrcv_l2.predict_proba(test_data)
lrcv_l2_probs = lrcv_l2.predict_proba(test_data)[:,1]
print('ROC-AUC score with l2 penalty:', roc_auc_score(test_label, lrcv_l2_probs))

ROC-AUC score with l2 penalty: 0.6992008578787977
Wall time: 1min


Using the liblinear solver, l1 penalty has a very slightly better ROC-AUC score.

Now, let's get the confusion matrix and precision/recall/F1-score.

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(test_label, lrcv_l1_pred)

array([[385, 206],
       [237, 391]], dtype=int64)

In [19]:
from sklearn.metrics import classification_report

print(classification_report(test_label, lrcv_l1_pred))

             precision    recall  f1-score   support

      False       0.62      0.65      0.63       591
       True       0.65      0.62      0.64       628

avg / total       0.64      0.64      0.64      1219



## Stochastic Gradient Descent

We will test SGD optimization with logistic regression and modified Huber loss functions. From the documentation (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier), "‘modified_huber’ is another smooth loss that brings tolerance to outliers as well as probability estimates". For each, we will test 'l2', 'l1', and 'elasticnet' penalties.

According to the documentation (http://scikit-learn.org/stable/modules/sgd.html#tips-on-practical-use), "finding a reasonable regularization term $\alpha$ is best done using GridSearchCV, usually in the range 10.0**-np.arange(1,7)."

In [20]:
# Importing SGDClassifier and GridSearchCV, creating alphas and penalties dictionaries for use with GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

alphas = {'alpha': 10.0**-np.arange(1,7)}
penalties = {'l2': 0, 'l1': 1, 'elasticnet': 0.5}

### SGD with log loss

In [21]:
%%time
# Using GridSearchCV to find optimal value for alpha for each penalty type, using ROC-AUC score as with other models

gs_log_scores = {}
gs_log_param = {}

for k,v in penalties.items():
    print('\nPenalty:', k, '\tElastic Net mixing parameter:', v)
    gs_log = GridSearchCV(SGDClassifier(loss='log', penalty=k, l1_ratio=v, random_state=10), param_grid=alphas,
                          scoring='roc_auc', cv=10, verbose=3)
    gs_log_fit = gs_log.fit(train_data, train_label)
    score = gs_log.score(test_data, test_label)
    gs_log_scores[k] = score
    param = gs_log.best_params_
    gs_log_param[k] = param
    print('ROC-AUC score on test set for', k, 'penalty: %f' % score)
    print('Best alpha for', k, 'penalty: %s' % param)

print('\nFinal results:')
print(gs_log_param, gs_log_scores)


Penalty: l2 	Elastic Net mixing parameter: 0
Fitting 10 folds for each of 6 candidates, totalling 60 fits
[CV] alpha=0.1 .......................................................
[CV] ............... alpha=0.1, score=0.561143586642478, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.5670395753544312, total=   0.0s
[CV] alpha=0.1 .......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and defa

[CV] .............. alpha=0.1, score=0.6054390915809984, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.5568097829738627, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6212625142780352, total=   0.0s
[CV] alpha=0.1 .......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] .............. alpha=0.1, score=0.6486085343228202, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6466857817507169, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.6347782088041829, total=   0.0s
[CV] alpha=0.1 .......................................................


...

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-06, score=0.6424354865913308, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6531624219935909, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6453213650605472, total=   0.0s
[CV] alpha=1e-06 .....................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-06, score=0.5503768312304175, total=   0.0s
ROC-AUC score on test set for elasticnet penalty: 0.679306
Best alpha for elasticnet penalty: {'alpha': 0.01}

Final results:
{'l2': {'alpha': 0.0001}, 'l1': {'alpha': 0.01}, 'elasticnet': {'alpha': 0.01}} {'l2': 0.6164468082813325, 'l1': 0.6381955446344855, 'elasticnet': 0.679305829480423}
Wall time: 14.7 s


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    4.9s finished
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [22]:
log_penalty = max(gs_log_scores, key=lambda key: gs_log_scores[key])
print('Best performance for SGD with log loss ->', log_penalty, '\nROC-AUC score:', gs_log_scores[log_penalty],
      '\nalpha:', gs_log_param[log_penalty])

Best performance for SGD with log loss -> elasticnet 
ROC-AUC score: 0.679305829480423 
alpha: {'alpha': 0.01}


Using the best performer to get predicted values and other evaluation metrics:

In [23]:
sgd_log_best = SGDClassifier(loss='log', penalty=log_penalty, alpha=gs_log_param[log_penalty].get('alpha'), 
                             l1_ratio=penalties[log_penalty], random_state=10)
sgd_log_best.fit(train_data, train_label)
sgd_log_pred = sgd_log_best.predict(test_data)

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [24]:
sgd_log_pred

array([ True,  True, False, ..., False, False, False])

In [25]:
confusion_matrix(test_label, sgd_log_pred)

array([[548,  43],
       [501, 127]], dtype=int64)

In [26]:
print(classification_report(test_label, sgd_log_pred))

             precision    recall  f1-score   support

      False       0.52      0.93      0.67       591
       True       0.75      0.20      0.32       628

avg / total       0.64      0.55      0.49      1219



### SGD with modified Huber loss

In [27]:
%%time
# Using GridSearchCV to find optimal value for alpha for each penalty type, using ROC-AUC score as with other models

gs_hub_scores = {}
gs_hub_param = {}

for k,v in penalties.items():
    print('\nPenalty:', k, '\tElastic Net mixing parameter:', v)
    gs_hub = GridSearchCV(SGDClassifier(loss='modified_huber', penalty=k, l1_ratio=v, random_state=10), param_grid=alphas,
                          scoring='roc_auc', cv=10, verbose=3)
    gs_hub_fit = gs_hub.fit(train_data, train_label)
    score_hub = gs_hub.score(test_data, test_label)
    gs_hub_scores[k] = score_hub
    param_hub = gs_hub.best_params_
    gs_hub_param[k] = param_hub
    print('ROC-AUC score on test set for', k, 'penalty: %f' % score_hub)
    print('Best alpha for', k, 'penalty: %s' % param_hub)

print('\nFinal results:')
print(gs_hub_param, gs_hub_scores)


Penalty: l2 	Elastic Net mixing parameter: 0
Fitting 10 folds for each of 6 candidates, totalling 60 fits
[CV] alpha=0.1 .......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] .............. alpha=0.1, score=0.5516864879392596, total=   0.0s
[CV] alpha=0.1 .......................................................
[CV] .............. alpha=0.1, score=0.5659141302156823, total=   0.0s
[CV] alpha=0.1 .......................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and defa

[CV] .............. alpha=0.1, score=0.6114022710475039, total=   0.0s
[CV] alpha=0.1 .......................................................


...

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-06, score=0.6159040516024995, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6412211165457918, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6524540394670264, total=   0.0s
[CV] alpha=1e-06 .....................................................


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
C:\Users\Miranda\Anaconda3\l

[CV] ............ alpha=1e-06, score=0.6531961544948558, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.6453213650605472, total=   0.0s
[CV] alpha=1e-06 .....................................................
[CV] ............ alpha=1e-06, score=0.5503937674654924, total=   0.0s
ROC-AUC score on test set for elasticnet penalty: 0.679640
Best alpha for elasticnet penalty: {'alpha': 0.1}

Final results:
{'l2': {'alpha': 0.01}, 'l1': {'alpha': 0.01}, 'elasticnet': {'alpha': 0.1}} {'l2': 0.6333241725672778, 'l1': 0.6459471693232888, 'elasticnet': 0.6796399280071562}
Wall time: 14.3 s


C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    4.8s finished
C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol wi

In [28]:
hub_penalty = max(gs_hub_scores, key=lambda key: gs_hub_scores[key])
print('Best performance for SGD with modified Huber loss ->', hub_penalty, '\nROC-AUC score:', gs_hub_scores[hub_penalty],
      '\nalpha:', gs_hub_param[hub_penalty])

Best performance for SGD with modified Huber loss -> elasticnet 
ROC-AUC score: 0.6796399280071562 
alpha: {'alpha': 0.1}


Using the best performer to get predicted values and other evaluation metrics:

In [29]:
sgd_hub_best = SGDClassifier(loss='modified_huber', penalty=hub_penalty, alpha=gs_hub_param[hub_penalty].get('alpha'),
                             l1_ratio=penalties[hub_penalty], random_state=10)
sgd_hub_best.fit(train_data, train_label)
sgd_hub_pred = sgd_hub_best.predict(test_data)

C:\Users\Miranda\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [30]:
sgd_hub_pred

array([ True,  True, False, ..., False, False, False])

In [31]:
confusion_matrix(test_label, sgd_hub_pred)

array([[508,  83],
       [430, 198]], dtype=int64)

In [32]:
print(classification_report(test_label, sgd_hub_pred))

             precision    recall  f1-score   support

      False       0.54      0.86      0.66       591
       True       0.70      0.32      0.44       628

avg / total       0.63      0.58      0.55      1219

